## Get Data

In [49]:
%matplotlib inline
import matplotlib.pyplot as plt
import fitbit

import gather_keys_oauth2 as Oauth2
import pandas as pd 
from datetime import datetime as d


CLIENT_ID='23RPGF'
CLIENT_SECRET='03d2a21d88b09241a0d06db073bca7c1'

## API Authorization

In [50]:
server=Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN=str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN=str(server.fitbit.client.session.token['refresh_token'])
auth2_client=fitbit.Fitbit(CLIENT_ID,CLIENT_SECRET,oauth2=True,access_token=ACCESS_TOKEN,refresh_token=REFRESH_TOKEN)

[05/Jan/2024:12:17:21] ENGINE Listening for SIGTERM.
[05/Jan/2024:12:17:21] ENGINE Bus STARTING
[05/Jan/2024:12:17:21] ENGINE Set handler for console events.
CherryPy Checker:
The Application mounted at '' has an empty config.

[05/Jan/2024:12:17:21] ENGINE Started monitor thread 'Autoreloader'.
[05/Jan/2024:12:17:22] ENGINE Serving on http://127.0.0.1:8080
[05/Jan/2024:12:17:22] ENGINE Bus STARTED


127.0.0.1 - - [05/Jan/2024:12:17:23] "GET /?code=41ee6a07db3a3eb3eaf1c262b77b7936f3a0bc19&state=sGOjdVGAydvhAwgiaj9wGkfYCWULMH HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"


[05/Jan/2024:12:17:24] ENGINE Bus STOPPING
[05/Jan/2024:12:17:24] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[05/Jan/2024:12:17:24] ENGINE Removed handler for console events.
[05/Jan/2024:12:17:24] ENGINE Stopped thread 'Autoreloader'.
[05/Jan/2024:12:17:24] ENGINE Bus STOPPED
[05/Jan/2024:12:17:24] ENGINE Bus EXITING
[05/Jan/2024:12:17:24] ENGINE Waiting for child threads to terminate...
[05/Jan/2024:12:17:24] ENGINE Bus EXITED
[05/Jan/2024:12:17:24] ENGINE Waiting for thread Thread-46.


## 5a.) Get One day of Data

In [51]:
oneDate = d(2023, 12, 18)

In [52]:
help(auth2_client.intraday_time_series)

Help on method intraday_time_series in module fitbit.api:

intraday_time_series(resource, base_date='today', detail_level='1min', start_time=None, end_time=None) method of fitbit.api.Fitbit instance
    The intraday time series extends the functionality of the regular time series, but returning data at a
    more granular level for a single day, defaulting to 1 minute intervals. To access this feature, one must
    fill out the Private Support form here (see https://dev.fitbit.com/docs/help/).
    For details on the resources available and more information on how to get access, see:
    
    https://dev.fitbit.com/docs/activity/#get-activity-intraday-time-series



In [53]:
oneDayData = auth2_client.intraday_time_series('activities/heart',
                                               base_date=oneDate,
                                               detail_level='1sec')

In [54]:
df = pd.DataFrame(oneDayData['activities-heart-intraday']['dataset'])

In [55]:
df.head()

,time,value
0,13:08:38,70
1,13:08:53,70
2,13:09:03,70
3,13:09:13,73
4,13:09:18,74


In [56]:
df['time'] = pd.to_datetime(df['time'])

df.set_index('time', inplace=True)

df_resampled = df.resample('S').ffill()

df_resampled.reset_index(inplace=True)

C:\Users\davi-\AppData\Local\Temp\ipykernel_10620\1680930436.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time'])


In [57]:
df_resampled['time'] = df_resampled['time'].dt.time

In [58]:
file_path = 'full_dataset/heart_rate.csv'

df_resampled.to_csv(file_path, index=False)

In [64]:
song_lengths = {
    "Die Young Kesha": "3:32",
    "Chopin Opus 9 No 2": "4:29",
    "Bangerang Skrillex": "3:38",
    "Slipknot psychosocial": "5:02"
}

for song, length in song_lengths.items():
    mins, secs = map(int, length.split(':'))
    song_lengths[song] = pd.Timedelta(minutes=mins, seconds=secs)


start_times = {
    "Martin": ["13:49:00", "13:58:00", "14:03:00", "14:08:00"],
    "Daniel": ["14:17:00", "14:21:00", "14:26:00", "14:31:00"],
    "Timm": ["14:39:00", "14:43:00", "14:49:05", "14:54:00"],
    "Steffen": ["15:01:00", "15:05:00", "15:10:00", "15:14:00"]
}


for person, times in start_times.items():
    start_times[person] = [pd.to_datetime(t) for t in times]


dfs = {}

for person, times in start_times.items():
    for i, start in enumerate(times):
        song_name = list(song_lengths.keys())[i]
        end = start + song_lengths[song_name]
        time_range = pd.date_range(start=start, end=end, freq='S')
        df = pd.DataFrame(time_range, columns=['time'])
        df['person'] = person
        df['song'] = song_name
        df_name = f"{person}_{song_name.replace(' ', '_')}"
        dfs[df_name] = df

        
dfs[list(dfs.keys())[0]].head()

,time,person,song
0,2024-01-05 13:49:00,Martin,Die Young Kesha
1,2024-01-05 13:49:01,Martin,Die Young Kesha
2,2024-01-05 13:49:02,Martin,Die Young Kesha
3,2024-01-05 13:49:03,Martin,Die Young Kesha
4,2024-01-05 13:49:04,Martin,Die Young Kesha


In [72]:
import os

# Load the heart rate data
heart_rate_df = pd.read_csv('full_dataset/heart_rate.csv')

heart_rate_df['time'] = pd.to_datetime(heart_rate_df['time'])

heart_rate_df.set_index('time', inplace=True)


df_resampled = heart_rate_df.resample('S').ffill()


df_resampled.reset_index(inplace=True)

C:\Users\davi-\AppData\Local\Temp\ipykernel_10620\2492505782.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  heart_rate_df['time'] = pd.to_datetime(heart_rate_df['time'])


In [73]:
song_lengths = {
    "Die Young Kesha": "3:32",
    "Chopin Opus 9 No 2": "4:29",
    "Bangerang Skrillex": "3:38",
    "Slipknot psychosocial": "5:02"
}

start_times = {
    "Martin": ["13:49:00", "13:58:00", "14:03:00", "14:08:00"],
    "Daniel": ["14:17:00", "14:21:00", "14:26:00", "14:31:00"],
    "Timm": ["14:39:00", "14:43:00", "14:49:05", "14:54:00"],
    "Steffen": ["15:01:00", "15:05:00", "15:10:00", "15:14:00"]
}

In [74]:
for song, length in song_lengths.items():
    mins, secs = map(int, length.split(':'))
    song_lengths[song] = pd.Timedelta(minutes=mins, seconds=secs)

for person, times in start_times.items():
    start_times[person] = [pd.to_datetime(t) for t in times]

base_path = 'data/'

for person, times in start_times.items():
    person_dir = os.path.join(base_path, person)
    os.makedirs(person_dir, exist_ok=True)
    
    for i, start in enumerate(times):
        song_name = list(song_lengths.keys())[i]
        end = start + song_lengths[song_name]
        # Extract the segment from the resampled heart rate data
        mask = (df_resampled['time'] >= start) & (df_resampled['time'] <= end)
        song_df = df_resampled.loc[mask]
        # Save the segment to a CSV file
        csv_path = os.path.join(person_dir, f"{song_name.replace(' ', '_')}.csv")
        song_df.to_csv(csv_path, index=False, columns=['time', 'value'])
